# [Internal] CreateMesh Demo


#### Import the SDK

We're going to point it at the main int environment.


In [1]:
import luminarycloud as lc

# <internal>
# Point the default client to main (Internal use only)
# Please keep the <internal> tags because assistant will use it to filter out the internal code.
lc.set_default_client(
    lc.Client(
        target="apis.main.int.luminarycloud.com",
        domain="luminarycloud-dev.us.auth0.com",
        client_id="mbM8OSEk5ShoU5iKfzUxSinKluPlxGQ9",
        audience="https://api-dev.luminarycloud.com",
    )
)
# </internal>

print(f"SDK version: {lc.__version__}")
print(f"SDK installation location: {lc.__file__}")

SDK version: 0.8.2
SDK installation location: /home/gavin_luminarycloud_com/venvs/sdk-3.10/lib/python3.10/site-packages/luminarycloud/__init__.py


#### Create a project and upload a geometry


In [2]:
project = lc.create_project(name="SDK CreateMesh Example")

Failed to silently authenticate.
Traceback (most recent call last):
  File "/home/gavin_luminarycloud_com/venvs/sdk-3.10/lib/python3.10/site-packages/luminarycloud/_auth/auth.py", line 152, in silently_renew_access_token
    response.raise_for_status()
  File "/home/gavin_luminarycloud_com/venvs/sdk-3.10/lib/python3.10/site-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://luminarycloud-dev.us.auth0.com/oauth/token


Listening on port 10001...
Interactive login required. Your browser has been opened to visit the following URL:

 https://luminarycloud-dev.us.auth0.com/authorize?audience=https%3A%2F%2Fapi-dev.luminarycloud.com&client_id=mbM8OSEk5ShoU5iKfzUxSinKluPlxGQ9&code_challenge=NEMX-eD3LDbf3EJHldmdCx2J6nKF4T85uHIKk2H-dxg&code_challenge_method=S256&response_type=code&scope=openid+profile+email+offline_access&state=A_jTq61DA4duj2j0NqTvc4xjB85V_hGtiYYdBVWwXwE&redirect_uri=http%3A%2F%2F127.0.0.1%3A10001%2Fcallback 



I0000 00:00:1735841057.261895 2554159 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


In [3]:
geometry = project.create_geometry("../../../testdata/cad/two_cubes.x_b", wait=True)
geometry.check()

/home/gavin_luminarycloud_com/venvs/sdk-3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.int.luminarycloud.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/gavin_luminarycloud_com/venvs/sdk-3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.int.luminarycloud.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/gavin_luminarycloud_com/venvs/sdk-3.10/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'main.int.luminarycloud.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthe

(True, [])

#### Load geometry to setup


In [4]:
project.load_geometry_to_setup(geometry)

We can take a look at the IDs of the faces and bodies in the geometry.


In [7]:
all_surfaces, all_volumes = geometry.list_entities()
print("Surfaces: ", [surface.id for surface in all_surfaces])
print("Volumes: ", [volume.id for volume in all_volumes])

Surfaces:  ['0/bound/BC_2', '0/bound/BC_3', '0/bound/BC_4', '0/bound/BC_5', '0/bound/BC_6', '0/bound/BC_7', '1/bound/BC_8', '1/bound/BC_9', '1/bound/BC_10', '1/bound/BC_11', '1/bound/BC_12', '1/bound/BC_13']
Volumes:  ['0', '1']


Let's go ahead and mesh it.


In [17]:
# from luminarycloud._proto.api.v0.luminarycloud.mesh.mesh_pb2 import MeshGenerationParams

# gen_params = MeshGenerationParams(
#     geometry_id=geometry.id,
#     proximity_layers=1,
#     mesh_complexity_params=MeshGenerationParams.MeshComplexityParams(
#         type=MeshGenerationParams.MeshComplexityParams.ComplexityType.MAX,
#         limit_max_cells=int(5e7),
#     ),
#     volume_params=[
#         MeshGenerationParams.VolumeParams(
#             min_size=0.4,
#             max_size=0.5,
#             volumes=[body.id for body in bodies], # All volumes
#         ),
#     ],
#     model_params=[
#         MeshGenerationParams.ModelParams(
#             curvature=8,
#             max_size=0.5,
#             surfaces=[face.id for face in faces], # All surfaces
#         ),
#     ],
#     bl_params=[
#         MeshGenerationParams.BoundaryLayerParams(
#             n_layers=40,
#             initial_size=1e-6,
#             growth_rate=1.2,
#             surfaces=[face.id for face in faces] # All surfaces
#         ),
#     ],
# )

#### Creating a minimal mesh

Formerly, in order to kick off a mesh adaptation
workflow in the SDK, we would first have needed to generate a minimal mesh
via the UI.

This lets us do it entirely within the SDK.


In [5]:
minimal_mesh_params = lc.meshing.MeshGenerationParams(
    geometry_id=geometry.id,
    sizing_strategy=lc.meshing.sizing_strategy.Minimal(),
)
minimal_mesh = project.create_or_get_mesh(minimal_mesh_params, name="minimal mesh")

#### Creating a regular old mesh

This is for a more traditional workflow.

We just need to specify max cell count, and size constraints for the mesh elements.


In [8]:
mesh_params = lc.meshing.MeshGenerationParams(
    geometry_id=geometry.id,
    sizing_strategy=lc.meshing.sizing_strategy.MaxCount(int(5e7)),
    min_size=0.4,
    max_size=0.5,
)

# To add boundary layers for surfaces.
mesh_params.boundary_layer_params.append(
    lc.meshing.BoundaryLayerParams(
        surfaces=all_surfaces,
        # You can optionally override the default values for the boundary layer params.
        # n_layers=40,
        # initial_size=1e-6,
        # growth_rate=1.2,
    )
)

mesh = project.create_or_get_mesh(mesh_params, name="meshy mesh")

Yes, tags do get unrolled, but they need to be prefixed with `lcTag/tagContainer`.


In [14]:
project.delete()